In [2]:
from flask import Flask, request, send_from_directory
from linebot.v3.messaging import Configuration, MessagingApi, ApiClient, ReplyMessageRequest, TextMessage, ImageMessage
from linebot.v3.webhooks import WebhookHandler
from linebot.v3.webhooks.models import MessageEvent, ImageMessageContent
from detect_yolo import detect_image
import config
import os
import requests

app = Flask(__name__)
configuration = Configuration(access_token=config.LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(config.LINE_CHANNEL_SECRET)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    handler.handle(body, signature)
    return 'OK'

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_image(event):
    message_id = event.message.id
    image_content = download_image(message_id)
    image_path = f'temp_{message_id}.jpg'
    with open(image_path, 'wb') as f:
        f.write(image_content)
    result_path, result_data = detect_image(image_path)
    os.remove(image_path)

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text="ผลการวินิจฉัยเบื้องต้นจากภาพที่ส่งมา:"),
                    ImageMessage(original_content_url=full_url(result_path),
                                 preview_image_url=full_url(result_path))
                ]
            )
        )

def full_url(local_path):
    return request.host_url + local_path

def download_image(message_id):
    headers = {"Authorization": f"Bearer {config.LINE_CHANNEL_ACCESS_TOKEN}"}
    url = f"https://api-data.line.me/v2/bot/message/{message_id}/content"
    res = requests.get(url, headers=headers)
    return res.content

@app.route('/static/results/<filename>')
def serve_result(filename):
    return send_from_directory('static/results', filename)

if __name__ == "__main__":
    app.run(port=5000)

ImportError: cannot import name 'WebhookHandler' from 'linebot.v3.webhooks' (/opt/anaconda3/lib/python3.12/site-packages/linebot/v3/webhooks/__init__.py)